# Lichen Likelihood Project: Expectation Maximization

In [15]:
import pandas as pd
import numpy as np

In [16]:
unpickled_df = pd.read_pickle('element_analysis.pkl') 

FileNotFoundError: [Errno 2] No such file or directory: 'element_analysis.pkl'

In [ ]:
# print columns ending in "binned"
unpickled_df.columns[unpickled_df.columns.str.endswith('binned')]

Index(['Nitrogen (% dw)_binned', 'Sulfur (% dw)_binned',
       'Phosphorous (ppm dw)_binned', 'Lead (ppm dw)_binned',
       'Copper (ppm dw)_binned', 'Chromium (ppm dw)_binned',
       'Year of tissue collection_binned', 'Air pollution score_binned',
       'Region_binned',
       'Code for scientific name and authority in lookup table_binned'],
      dtype='object')

In [14]:
# Helpers
df = unpickled_df.copy()
def idx_map_from_series(s):
    """Return dict mapping original category -> index and reverse list"""
    cats = list(pd.Categorical(s).categories)
    mapping = {c:i for i,c in enumerate(cats)}
    return mapping, cats

def one_hot_index_from_value(mapping, val):
    return mapping[val]

def normalize_rows(mat, axis=-1):
    s = mat.sum(axis=axis, keepdims=True)
    s[s == 0] = 1.0
    return mat / s

def logsumexp(a, axis=None):
    a_max = np.max(a, axis=axis, keepdims=True)
    res = a_max + np.log(np.sum(np.exp(a - a_max), axis=axis, keepdims=True))
    if axis is not None:
        return np.squeeze(res, axis=axis)
    return res

In [ ]:

class BayesianNetworkEM:
    def __init__(self, hidden_states=2, pollution_latent=True, seed=0):
        """
        Initializes CPT tables with random values consistent with the BN structure.
        """
        np.random.seed(seed)
        self.df = df.copy().reset_index(drop=True)
        self.N = len(df)
        self.H = hidden_states # need at least 2 latent classes 
        self.pollution_latent = pollution_latent
        self.elements = [
            'Nitrogen (% dw)_binned',
            'Sulfur (% dw)_binned',
            'Phosphorous (ppm dw)_binned',
            'Lead (ppm dw)_binned',
            'Copper (ppm dw)_binned',
            'Chromium (ppm dw)_binned'
        ]
        self.year_col = 'Year of tissue collection_binned'
        self.pollution_col = 'Air pollution score_binned'
        self.region_col = 'Region_binned'
        self.species_col = 'Code for scientific name and authority in lookup table_binned'
        self.maps = {}
        self.rev = {}
        cols_for_map = self.elements + [self.year_col, self.pollution_col, self.region_col, self.species_col]
        for col in cols_for_map:
            mapping, cats = idx_map_from_series(self.df[col])
            self.maps[col] = mapping
            self.rev[col] = cats
        self.K_region = len(self.rev[self.region_col])
        self.K_year = len(self.rev[self.year_col])
        self.K_poll = len(self.rev[self.pollution_col])
        self.K_species = len(self.rev[self.species_col])
        self.K_elem = {col: len(self.rev[col]) for col in self.elements}

        # Initialize CPTs as probabilities, not log probs
        self.initialize_random_CPTs()

    def initialize_random_CPTs(self):
            """
            Randomly initialize all CPTs:
            Ensure each CPT is normalized over its conditional domain.
            """
            # P(H | Region, Year)
            self.P_H_given_RY = np.random.rand(self.K_region, self.K_year, self.H)
            self.P_H_given_RY = normalize_rows(self.P_H_given_RY, axis=-1)
            # P(Pe | H)
            self.P_Pe_given_H = np.random.rand(self.H, self.K_poll)
            self.P_Pe_given_H = normalize_rows(self.P_Pe_given_H, axis=-1)
            # P(Sp | H, Pe)
            self.P_Sp_given_HP = np.random.rand(self.H, self.K_poll, self.K_species)
            self.P_Sp_given_HP = normalize_rows(self.P_Sp_given_HP, axis=-1)
            # For each element: P(Elem | Sp, Pe)
            self.P_elem_given_SP = {}
            for col in self.elements:
                self.P_elem_given_SP[col] = np.random.rand(self.K_species, self.K_poll, self.K_elem[col])
                self.P_elem_given_SP[col] = normalize_rows(self.P_elem_given_SP[col], axis=-1)

    def e_step(self):
        """
        Performs the E-step
            For each sample n:
                Compute posterior responsibilities
            Store in self.gamma with shape [num_samples, num_hidden_states]
        """


    def compute_posterior_hidden_probs(self, sample_row):
        """
        Compute P(H=h | observed sample features) for a single sample
        Returns a vector normalized over hidden states
        """


    def m_step(self):
        """
        Performs the M-step:
            Update CPTs:
                - P(H | Region, FieldDate)
                - P(Pollution | H)
                - P(Species | H, Pollution)
                - P(Element_i_bucket | Species, Pollution) for each element node
        Uses expected counts
        """


    def update_hidden_CPT(self):
        """
        Update P(H | Region, FieldDate)
        Accumulate weighted counts per condition using responsibilities calculated above
        """


    def update_pollution_CPT(self):
        """
        Update P(Pollution | H)
        Pollution is observed, responsibilities provides the weighting.
        """


    def update_species_CPT(self):
        """
        Update: P(Species | H, Pollution)
        Species is observed, conditioned on hidden state and pollution bucket(S)
        """


    def update_element_bucket_CPTs(self):
        """
        For each tissue element bucket node:
            Update: P(ElementBucket | Species, Pollution)
        Summed over hidden responsibilities since element buckets have no H parent
        """


    def log_likelihood(self):
        """
        Compute the complete-data log likelihood using current CPTs and responsibilities
        Used for eval
        """


    def run(self, max_iters=50, tol=1e-6):
        """
        Full EM loop:
        initialize_random_CPTs()
        e_step()
        m_step()
        compute log likelihood and check tolerance
        Repeat.
        Returns learned CPT parameters.
        """
    def predict_pollution_distribution(self, row):
        """
        Compute P(Pollution | observed row)
        using:
            P(P | H) * P(H | row)

        return numpy array of num_pollution_values
        """
        pass



In [ ]:
bnem= BayesianNetworkEM(hidden_states=2, pollution_latent=True, seed=42)